In [1]:
import logging
from datetime import datetime, timezone
from typing import List

from redis.asyncio import Redis

from app.database_redis.connection import get_redis_client
from app.database_redis.dals.connection_dal import ConnectionDAL
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.apis.streamqueue_service.schemas import AudioChunkInfo, ExistingConnectionInfo
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings

logger = logging.getLogger(__name__)


check_and_process_connections_interval_sec: 2.0


In [2]:
from app.services.audio.audio import AudioFileCorruptedError, AudioSlicer

In [3]:
import pandas as pd

/tmp/ipykernel_58265/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [51]:
connection_id = 'test1'

In [52]:
path = f'/audio/{connection_id}.webm'

In [53]:
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 30)
slice_duration = audio_slicer.audio.duration_seconds
audio_data = await audio_slicer.export_data()

2024-06-22 12:31:03,888 - INFO - app.services.audio.audio - start: None


In [54]:
audio_slicer.audio

In [56]:
meeting_id = 'test2'
user_id = 'ac43a16f-2a2c-41d7-99f7-035e4893c64f'

In [57]:
segment_start_timestamp = pd.Timestamp.utcnow()
segment_end_timestamp = segment_start_timestamp + pd.Timedelta(seconds = 360)

In [58]:
current_time = datetime.now(timezone.utc)

In [59]:
connection = Connection(redis_client, connection_id, user_id)
await connection.update_timestamps(segment_start_timestamp, segment_end_timestamp)

In [60]:
connection.start_timestamp

Timestamp('2024-06-22 12:31:12.109718+0000', tz='UTC')

In [61]:
connection.end_timestamp

Timestamp('2024-06-22 12:37:12.109718+0000', tz='UTC')

In [62]:
segment_start_timestamp

Timestamp('2024-06-22 12:31:12.109718+0000', tz='UTC')

In [63]:
segment_end_timestamp

Timestamp('2024-06-22 12:37:12.109718+0000', tz='UTC')

In [64]:
meeting = Meeting(redis_client, meeting_id)

await meeting.load_from_redis()
await meeting.add_connection(connection.id)
await meeting.set_start_timestamp(segment_start_timestamp)

In [65]:
meeting.diarizer_last_updated_timestamp = meeting.diarizer_last_updated_timestamp or segment_start_timestamp
meeting.transcriber_last_updated_timestamp = (
    meeting.transcriber_last_updated_timestamp or segment_start_timestamp
)

In [66]:

print("diarizer added")
diarizer = Diarizer(redis_client)
await diarizer.add_todo(meeting.meeting_id)
await meeting.update_diarizer_timestamp(
    segment_start_timestamp, diarizer_last_updated_timestamp=connection.start_timestamp
)

diarizer added


In [67]:

print("transcriber added")
transcriber = Transcriber(redis_client)
await transcriber.add_todo(meeting.meeting_id)
await meeting.update_transcriber_timestamp(
    segment_start_timestamp, transcriber_last_updated_timestamp=connection.start_timestamp
)

transcriber added


In [77]:
connection.start_timestamp

datetime.datetime(2024, 5, 30, 11, 25, 15, 477541, tzinfo=tzutc())

In [63]:
meeting.meeting_id

'test'

In [43]:
current_time

datetime.datetime(2024, 6, 22, 11, 11, 47, 420783, tzinfo=datetime.timezone.utc)

In [ ]:
await meeting.update_redis()